In [1]:
import numpy as np
import pandas as pd

import findspark
findspark.init()
from pyspark import SparkContext

sc = SparkContext("local[*]", "Name of the Program")

25/02/24 16:09:00 WARN Utils: Your hostname, jalvarez resolves to a loopback address: 127.0.1.1; using 192.168.1.51 instead (on interface wlo1)
25/02/24 16:09:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/24 16:09:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Ruta del archivo CSV
archivo_csv = "data_ok.csv"

# Leer el archivo como un RDD
rdd = sc.textFile(archivo_csv)
total_filas = float(rdd.count())
total_filas

5000.0

## Load data

## Ej1

In [3]:
rdd_38 = rdd.map(lambda x : float(x.split(" ")[38]))
suma = rdd_38.reduce(lambda x,y: x+y)
media = suma / total_filas

rdd_2 = rdd_38.map(lambda x: (x, media))

restas = rdd_2.map(lambda x: (x[0]-x[1])**2)
sumas = restas.reduce(lambda x,y: x+y)
varianza = sumas / total_filas

print(f'Medias: {media}. Varianzas: {varianza}')
print(f'Real. Media: {rdd_38.mean()}. Varianza: {rdd_38.stdev()**2}')

Medias: 1058.8409692255857. Varianzas: 3760978.8000936317
Real. Media: 1058.8409692255827. Varianza: 3760978.800093644


## Ej 2

In [4]:
rdd_numpy = rdd.map(lambda x : np.array(x.split(" "), dtype = float))
suma = rdd_numpy.reduce(lambda x,y: x+y)
media = suma / total_filas

rdd_2 = rdd_numpy.map(lambda x: (x, media))

restas = rdd_2.map(lambda x: (x[0]-x[1])**2)
sumas = restas.reduce(lambda x,y: x+y)
varianza = sumas / total_filas


#Se imprimen los 3 primeros para comprobar
print(f'Medias: {media[:3]}. Varianzas: {varianza[:3]}')
print(f'Real. Media: {rdd_numpy.mean()[:3]}. Varianza: {(rdd_numpy.stdev()**2)[:3]}')


Medias: [ 427.8384 8544.5638 1006.7712]. Varianzas: [4.01818166e+05 3.16837334e+09 2.95960564e+06]
Real. Media: [ 427.8384 8544.5638 1006.7712]. Varianza: [4.01818166e+05 3.16837334e+09 2.95960564e+06]


## Ej 3

In [5]:
# Transformar cada fila en una lista de tuplas (columna, valor)
rdd_tuplas = rdd.flatMap(lambda x: enumerate(x.split()))

# Convertir en un RDD de pares (columna, valor)
rdd_pares = rdd_tuplas.map(lambda x: (x[0], float(x[1])))

# Aplicar reduceByKey para sumar los valores por columna
rdd_reducido = rdd_pares.reduceByKey(lambda x, y: x + y)
 
rdd_media = rdd_reducido.map(lambda x: (x[0], (x[1]/total_filas)))

# Mostrar solo las primeras 5 filas en lugar de collect()
media = rdd_media.collect()
media=sorted(media,key=lambda x: x[0])

rdd_potencia = rdd_pares.map(lambda x: (x[0], ((x[1]-media[x[0]][1])**2)))
varianza = rdd_potencia.reduceByKey(lambda x, y: x + y)
std=varianza.map(lambda x: (x[0], (x[1]/total_filas)))
std=std.collect()
std=sorted(std,key=lambda x: x[0])
[(i, media[i][1], std[i][1] ) for i in range(len(media))]

[(0, 427.8384, 401818.16628544027),
 (1, 8544.5638, 3168373338.3683057),
 (2, 1006.7712, 2959605.6432505627),
 (3, 1409608.9152, 5883755024032.205),
 (4, 1.0, 0.0),
 (5, 6.6542, 2.3010223599999864),
 (6, 1460.0, 0.0),
 (7, 762.3126, 313332.59728123934),
 (8, 140.2394, 19828.36648764016),
 (9, 506626.9196, 345513253204.2377),
 (10, 28973.7058, 6514263.902446111),
 (11, 0.0108, 0.03228335999999504),
 (12, 0.9462, 0.05090556000000112),
 (13, 7.5006, 3.9563996399999914),
 (14, 1411.0728, 14631.804300159674),
 (15, 3307.1342, 19686496.994590227),
 (16, 198.7724, 103958.43979824062),
 (17, 835825.356, 12358118332394.264),
 (18, 43811.7826, 435068936.0129385),
 (19, 0.0, 0.0),
 (20, 238.93478482702486, 114088.27606048658),
 (21, 224802.51876693306, 151781717098.3398),
 (22, 16.71908086065675, 3110.345729247053),
 (23, 2519.3622, 19567004.17901126),
 (24, 0.0, 0.0),
 (25, 114.84695369961145, 105249.50629016865),
 (26, 36.99778951411964, 11237.074611699474),
 (27, 2531.919, 19550727.745639034),

## Ej 4

In [6]:
rdd_numpy = rdd.map(lambda x : np.array(x.split(" "), dtype = float))
suma = rdd_numpy.reduce(lambda x,y: x+y)
media = suma / total_filas

rdd_2 = rdd_numpy.map(lambda x: (x, media))

restas = rdd_2.map(lambda x: (x[0]-x[1])**2)
sumas = restas.reduce(lambda x,y: x+y)
desviacion = (sumas / total_filas)**0.5


#Se imprimen los 3 primeros para comprobar
print(f'Medias: {media[:3]}. Desviaciones: {desviacion[:3]}')
media = np.array(media)
desviacion = np.array(desviacion)
print(desviacion[:5])

Medias: [ 427.8384 8544.5638 1006.7712]. Desviaciones: [  633.89128901 56288.30552049  1720.35044199]
[6.33891289e+02 5.62883055e+04 1.72035044e+03 2.42564528e+06
 0.00000000e+00]


In [43]:
def f(x):
    a = (x - media) / desviacion
    return np.nan_to_num(a, copy=False,nan=0.0)

rdd_numpy = rdd.map(lambda x : np.array(x.split(" "), dtype = float))
normalizado = rdd_numpy.map(f)
normalizado.take(1)

/tmp/ipykernel_161711/1993936048.py:2: RuntimeWarning: invalid value encountered in divide


[array([-6.46543669e-01, -1.24689556e-01, -5.78237536e-01, -5.77673878e-01,
         0.00000000e+00,  2.27963108e-01,  0.00000000e+00, -9.33098977e-01,
        -7.26063865e-01, -7.83069037e-01,  8.86626584e-02, -6.01082925e-02,
         2.38451284e-01,  7.53819608e-01, -6.53699534e-01, -4.17885154e-01,
        -4.11792023e-01, -2.29312940e-01, -8.34764936e-01,  0.00000000e+00,
        -6.65795949e-01, -5.65116934e-01,  9.76279781e-01, -4.10620178e-01,
         0.00000000e+00,  1.13955413e-01,  7.30593884e-01, -4.13630931e-01,
         0.00000000e+00, -2.26263187e-03,  1.21784165e+00, -3.50959293e-01,
        -8.58088301e-02,  4.04643851e-01, -2.61479501e-01, -3.02573221e-01,
        -8.72831800e-01, -3.38737132e-01, -3.86134987e-01, -4.07374173e-01,
        -2.17656144e-01, -3.83624430e-01, -6.47131316e-01, -1.24899179e-01,
        -5.78021035e-01, -5.77400901e-01,  0.00000000e+00,  2.28448139e-01,
         0.00000000e+00, -9.43303713e-01, -7.33119410e-01, -7.84130337e-01,
         8.8